In [ ]:
import numpy as np
import pandas as pd
from functools import reduce
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupShuffleSplit, StratifiedKFold, GridSearchCV
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

#====================#
# 1️⃣ 读取并拼接三模态
#====================#
file_paths = {
    "smri": r"D:\!!yym\!!!ABCD_data\abcd-data-release-5.1_new\5.1processing\BrainAGE\smri_data.csv",
    "dm":   r"D:\!!yym\!!!ABCD_data\abcd-data-release-5.1_new\5.1processing\BrainAGE\dm_data.csv",
    "rsfmri": r"D:\!!yym\!!!ABCD_data\abcd-data-release-5.1_new\5.1processing\BrainAGE\rsfmri_data.csv"
}

data_dict = {}
common_cols = ["src_subject_id", "rel_family_id", "interview_age", "sex", "eventname"]

for modality, path in file_paths.items():
    df = pd.read_csv(path, na_values=[" ", "NA"])
    # 非 ID 列转成数值
    for col in df.columns:
        if col != "src_subject_id":
            df[col] = pd.to_numeric(df[col], errors='coerce')
    # 给特征加上模态前缀，避免重名
    feat_cols = [c for c in df.columns if c not in common_cols + ["group_id"]]
    df.rename(columns={c: f"{modality}_{c}" for c in feat_cols}, inplace=True)
    data_dict[modality] = df[common_cols + [f"{modality}_{c}" for c in feat_cols]]

# 按公共列横向合并三份数据
df_all = reduce(lambda left, right: pd.merge(left, right, on=common_cols, how="inner"),
                data_dict.values())
print("✅ 三模态数据已拼接完成！总样本量:", df_all.shape[0])

#====================#
# 2️⃣ Z-score 标准化脑指标
#====================#
brain_cols = [c for c in df_all.columns if c not in common_cols]
scaler = StandardScaler()
df_all[brain_cols] = scaler.fit_transform(df_all[brain_cols])
print("✅ 脑指标已标准化 (z-score)")

#====================#
# 3️⃣ 训练/测试集划分
#====================#
df_all["group_id"] = df_all["src_subject_id"].astype(str) + "_" + df_all["rel_family_id"].astype(str)
gss = GroupShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
for train_idx, test_idx in gss.split(df_all, groups=df_all["group_id"]):
    train_df = df_all.iloc[train_idx].copy()
    test_df  = df_all.iloc[test_idx].copy()

train_df["age_group"] = np.digitize(train_df["interview_age"], np.arange(100, 200, 10))
print(f"✅ 训练集: {train_df.shape[0]}  测试集: {test_df.shape[0]}")

#====================#
# 4️⃣ XGBoost 超参数搜索
#====================#
param_grid = {
    "max_depth": [4, 6, 8],
    "learning_rate": [0.01, 0.1, 0.2],
    "n_estimators": [500, 800, 1000]
}

features = [c for c in train_df.columns
            if c not in ["src_subject_id","rel_family_id","interview_age",
                         "group_id","sex","age_group","eventname"]]
X_train = train_df[features]
y_train = train_df["interview_age"]
age_group = train_df["age_group"]

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
xgb_model = xgb.XGBRegressor(objective="reg:squarederror")
grid_search = GridSearchCV(
    xgb_model, param_grid,
    cv=skf.split(X_train, age_group),
    scoring="neg_mean_absolute_error",
    n_jobs=1, verbose=1
)
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_
print("✅ 多模态最佳超参数:", grid_search.best_params_)

#====================#
# 5️⃣ 10 折回归校正系数
#====================#
beta_0_list, beta_1_list, beta_2_list = [], [], []
y_train_full = train_df["interview_age"]
sex_train = train_df["sex"]

for train_idx, val_idx in skf.split(X_train, np.digitize(y_train_full, np.arange(10, 100, 5))):
    X_fold_val = X_train.iloc[val_idx]
    y_fold_val = y_train_full.iloc[val_idx]
    sex_fold_val = sex_train.iloc[val_idx]

    y_pred_fold = best_model.predict(X_fold_val)

    reg = LinearRegression()
    reg.fit(np.column_stack((y_fold_val, sex_fold_val)), y_pred_fold)

    beta_0_list.append(reg.intercept_)
    beta_1_list.append(reg.coef_[0])
    beta_2_list.append(reg.coef_[1])

beta_0_final = np.mean(beta_0_list)
beta_1_final = np.mean(beta_1_list)
beta_2_final = np.mean(beta_2_list)
print(f"✅ 最终校正参数: β0={beta_0_final:.4f}, β1={beta_1_final:.4f}, β2={beta_2_final:.4f}")

#====================#
# 6️⃣ 测试集评估
#====================#
X_test = test_df[features]
y_test = test_df["interview_age"]
y_test_pred = best_model.predict(X_test)
print("✅ 测试集评估:",
      f"MAE={mean_absolute_error(y_test, y_test_pred):.4f}, ",
      f"RMSE={np.sqrt(mean_squared_error(y_test, y_test_pred)):.4f}, ",
      f"R²={r2_score(y_test, y_test_pred):.4f}")

#====================#
# 7️⃣ 全体样本预测 & BAG
#====================#
features_all = features
y_all_pred = best_model.predict(df_all[features_all])

# 校正脑龄
y_all_corrected = (y_all_pred - beta_0_final - beta_2_final * df_all["sex"]) / beta_1_final
corrected_bag = y_all_corrected - df_all["interview_age"]

bag_results = pd.DataFrame({
    "src_subject_id": df_all["src_subject_id"],
    "eventname": df_all["eventname"],
    "chronological_age": df_all["interview_age"],
    "corrected_age": y_all_corrected,
    "corrected_bag": corrected_bag
})

output_path = r"D:\!!yym\!!!ABCD_data\abcd-data-release-5.1_new\5.1processing\BrainAGE\multimodal_corrected_bag(1).csv"
bag_results.to_csv(output_path, index=False)
print("✅ 多模态整合后的校正 BAG 结果已保存:", output_path)

✅ 三模态数据已拼接完成！总样本量: 17162


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


✅ 脑指标已标准化 (z-score)


C:\Users\15098\AppData\Local\Temp\ipykernel_13712\2668626086.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_all["group_id"] = df_all["src_subject_id"].astype(str) + "_" + df_all["rel_family_id"].astype(str)


✅ 训练集: 8602  测试集: 8560
Fitting 10 folds for each of 27 candidates, totalling 270 fits


In [ ]:
import joblib

# 保存模型和关键数据
joblib.dump(best_model, "best_model.pkl")
joblib.dump(train_df, "train_df.pkl")
joblib.dump(test_df, "test_df.pkl")
joblib.dump(features, "features.pkl")